# Carregamento dos pacotes e dados

Ilustração de uma carteira hipotética com 15 ativos, indicando o valor atual dos investimentos em cada ativo, bem como a fração destes no portfolio. O Dataframe também indica a fração ideal, configurado pelo gestor, e a diferença entre o real e o desejado.

In [11]:
using JuMP, GLPK, DataFrames, XLSX, CSV, Ipopt


df = XLSX.readdata("G:/Meu Drive/Investimentos.xlsx", "Investimentos", "C1:M16")
df = DataFrame(Any[@view df[2:end, i] for i in 1:size(df, 2)], Symbol.(df[1, :]))

select!(df, Not([:Qnt,:PM,:Variacao,:Diff_Inv,:Atual,:Investido]))

,Ativo,Real,Fatia,Ideal,Diff_Fatia
,Any,Any,Any,Any,Any
1,IJR,1125.4,0.0496,0.1,-0.0504
2,VUG,2504.64,0.1103,0.1,0.0103
3,IRBO,1078.32,0.0475,0.1,-0.0525
4,IGV,2192.55,0.0965,0.1,-0.0035
5,IEO,510.08,0.0225,0.05,-0.0275
6,VBK,1473.85,0.0649,0.1,-0.0351
7,SNSR,1896.5,0.0835,0.1,-0.0165
8,AMZN,1671.06,0.0736,0.04,0.0336
9,FB,1612.85,0.071,0.04,0.031


# Definição das variáveis e condições de aporte

Configuração das variáveis que entrarão na otimização, bem como os parâmetros relacionados ao aporte e se as mudanças no portfolio considerarão a possibilidade de vendas de ativos, para equilibrar o portfolio.

A primeira ilustração não levará em consideração a venda de ativos, para equilibrar o portfólio.

In [3]:
#----Variáveis
investido = df.Real
ideal = df.Ideal
conj_ativos = collect(1:(size(df)[1])) # qnt. de atributos
valor_aporte = 1000 #EM DÓLARES
vender = false;

# Problema de otimização

In [8]:
#----Otimização
function OPT_problem(investido,ideal,conj_ativos,valor_aporte, vender)
    @eval begin
    OPT = Model(Ipopt.Optimizer);
    set_silent(OPT) # O otimizador não imprime nada.

    #Definição das variáveis
    if vender == false
        @variable(OPT, aporte[i in conj_ativos] >= 0)
    else
        @variable(OPT, aporte[i in conj_ativos])
    end
    @variable(OPT, new_frac[i in conj_ativos] >= 0)
    @variable(OPT, abs_frac[i in conj_ativos] >= 0)
    # @variable(OPT, investir[i in conj_ativos], binary = true)

    #Restrições
    @constraint(OPT, new_frac_, sum(new_frac[i] for i in conj_ativos) == 1)
    @NLconstraint(OPT, frac_ideal[i in conj_ativos],
        (new_frac[i] * sum(investido[i]+aporte[i] for i in conj_ativos))
        <= (investido[i]+aporte[i]))
    @constraint(OPT, abs_1[i in conj_ativos], abs_frac[i] >= (ideal[i] - new_frac[i]))
    @constraint(OPT, abs_2[i in conj_ativos], abs_frac[i] >= -(ideal[i] - new_frac[i]))
    @constraint(OPT, new_frac_less_one[i in conj_ativos], new_frac[i] <= 1)
    @constraint(OPT, val_aporte, sum(aporte[i] for i in conj_ativos) == valor_aporte)
    @constraint(OPT, val_aporte_max[i in conj_ativos], aporte[i] <= valor_aporte)
    # @constraint(OPT, corretagem, sum(investir[i] for i in conj_ativos) <= 10)


    #Função objetivo
    @objective(OPT, Min, sum(abs_frac[i] for i in conj_ativos))
    # @objective(OPT, Min, sum(L[t] for t in T_L)/L_norm + α*sum(sum(s[j,t] for j in p) for t in T_B))

    optimize!(OPT)
    fo = JuMP.objective_value(OPT)
    aportee=[JuMP.value.(aporte)[CartesianIndex(i)] for i in 1:length(ideal)]
    new_fracc=[JuMP.value.(new_frac)[CartesianIndex(i)] for i in 1:length(ideal)]

end
end

# Solução

In [7]:
OPT_problem(investido,ideal,conj_ativos,valor_aporte, vender);
print("\nFunção objetivo: ", fo, "\n")
print("Vender: ", vender, "\n")
print("Aporte: \$ ", valor_aporte, "\n\n")

insertcols!(df, size(df)[2]+1, :Aporte => round.(aportee, digits = 3))

df.Fatia = round.(new_fracc; digits = 3)
df.Diff_Fatia = round.(df.Ideal - df.Fatia, digits = 3)
insertcols!(df, 3, :Investido => round.(df.Real + df.Aporte, digits = 3))
# df.Real = round.(df.Real + df.Aporte, digits = 3)
# insertcols!(df, 8, :Diff_New => round.(df.Ideal - df.Fatia, digits = 3))
print(df)

print("\nAporte: ",sum(df.Aporte))
print("\nDólares Investidos: ",sum(df.Investido))


Função objetivo: 0.36975856898128745
Vender: false
Aporte: $ 1000

15×7 DataFrame
 Row │ Ativo  Real     Investido  Fatia    Ideal  Diff_Fatia  Aporte  
     │ Any    Any      Float64    Float64  Any    Float64     Float64 
─────┼────────────────────────────────────────────────────────────────
   1 │ IJR    1125.4    1235.23     0.052  0.1         0.048  109.826
   2 │ VUG    2504.64   2504.64     0.106  0.1        -0.006    0.0
   3 │ IRBO   1078.32   1623.52     0.068  0.1         0.032  545.202
   4 │ IGV    2192.55   2194.68     0.093  0.1         0.007    2.127
   5 │ IEO    510.08     580.931    0.025  0.05        0.025   70.851
   6 │ VBK    1473.85   1699.72     0.072  0.1         0.028  225.871
   7 │ SNSR   1896.5    1916.64     0.081  0.1         0.019   20.144
   8 │ AMZN   1671.06   1671.06     0.07   0.04       -0.03     0.0
   9 │ FB     1612.85   1612.85     0.068  0.04       -0.028    0.0
  10 │ VYM    1082.6    1083.29     0.046  0.05        0.004    0.685
  11 │ SOX

Os ativos que estavam com composição superior a faixa ideal não tiveram aportes (nem venda), como o esperado. Estes seriam os aportes necessários para buscar equilibrar o portfolio. Próximas versões poderiam adicionar custo de corretagem, para evitar aportes pequenos, como observado no ativo "IGV".

# Aportes com vendas de ativo

In [12]:
#----Variáveis
investido = df.Real
ideal = df.Ideal
conj_ativos = collect(1:(size(df)[1])) # qnt. de atributos
valor_aporte = 1000 #EM DÓLARES
vender = true;

In [13]:
#Carregando dados novamente devido as alterações anteriores.
df = XLSX.readdata("G:/Meu Drive/Investimentos.xlsx", "Investimentos", "C1:M16")
df = DataFrame(Any[@view df[2:end, i] for i in 1:size(df, 2)], Symbol.(df[1, :]))
select!(df, Not([:Qnt,:PM,:Variacao,:Diff_Inv,:Atual,:Investido]))

#Chamando função de otimização
OPT_problem(investido,ideal,conj_ativos,valor_aporte, vender);

#Print de resultados
print("\nFunção objetivo: ", fo, "\n")
print("Vender: ", vender, "\n")
print("Aporte: \$ ", valor_aporte, "\n\n")

insertcols!(df, size(df)[2]+1, :Aporte => round.(aportee, digits = 3))

df.Fatia = round.(new_fracc; digits = 3)
df.Diff_Fatia = round.(df.Ideal - df.Fatia, digits = 3)
insertcols!(df, 3, :Investido => round.(df.Real + df.Aporte, digits = 3))
# df.Real = round.(df.Real + df.Aporte, digits = 3)
# insertcols!(df, 8, :Diff_New => round.(df.Ideal - df.Fatia, digits = 3))
print(df)

print("\nAporte: ",sum(df.Aporte))
print("\nDólares Investidos: ",sum(df.Investido))


Função objetivo: 0.04543567248414747
Vender: true
Aporte: $ 1000

15×7 DataFrame
 Row │ Ativo  Real     Investido  Fatia    Ideal  Diff_Fatia  Aporte    
     │ Any    Any      Float64    Float64  Any    Float64     Float64   
─────┼──────────────────────────────────────────────────────────────────
   1 │ IJR    1125.4    2125.38     0.09   0.1         0.01     999.978
   2 │ VUG    2504.64   2412.97     0.102  0.1        -0.002    -91.669
   3 │ IRBO   1078.32   2078.3      0.088  0.1         0.012    999.978
   4 │ IGV    2192.55   2412.6      0.102  0.1        -0.002    220.046
   5 │ IEO    510.08    1225.55     0.052  0.05       -0.002    715.467
   6 │ VBK    1473.85   2404.41     0.101  0.1        -0.001    930.559
   7 │ SNSR   1896.5    2411.89     0.102  0.1        -0.002    515.385
   8 │ AMZN   1671.06    991.322    0.042  0.04       -0.002   -679.738
   9 │ FB     1612.85    991.312    0.042  0.04       -0.002   -621.538
  10 │ VYM    1082.6    1227.63     0.052  0.05    

A possibilidade de vendas de ativo possibilita que o portfolio tenda a alcançar as frações desejadas pelo gestor. É uma ferramenta bem útil para quem trabalha com um portfolio com muitos ativos e que queira rebalancear a carteira ao longo do tempo, buscando um manejo adequado a cada ativo, ao longo do ciclo econômico.